In [211]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from scipy import linalg

### Controller Design (with vx, vy, vz)


In [212]:
# Mass
m = 0.046

# Principle moments of inertia
J_x = 2.11e-05
J_y = 2.13e-05
J_z = 3.59e-05

# Acceleration of gravity
g = 9.81

p_eq = [m, J_x, J_y, J_z]

In [213]:
def Rotate(alpha, beta, gamma):
    Rz = sym.Matrix([[sym.cos(alpha), -sym.sin(alpha), 0],
                 [sym.sin(alpha), sym.cos(alpha), 0],
                 [0, 0, 1]])

    Ry = sym.Matrix([[sym.cos(beta), 0, sym.sin(beta)],
                 [0, 1, 0],
                 [-sym.sin(beta), 0, sym.cos(beta)]])

    Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(gamma), -sym.sin(gamma)],
                 [0, sym.sin(gamma), sym.cos(gamma)]])

    R_VtoO = Rz * Ry * Rx

    return R_VtoO

def N_func(alpha, beta, gamma):
    Ninv = sym.Matrix([[sym.cos(beta)*sym.cos(gamma), -sym.sin(gamma), 0],
                       [sym.cos(beta)*sym.sin(gamma), sym.cos(gamma), 0],
                       [-sym.sin(beta), 0, 1]])

    # N = sym.simplify(Ninv.inv())
    N = Ninv.inv()
    return N

In [214]:
# Added vx,vy,vz as part of state variables
o_x, o_y, o_z, vx, vy, vz, alpha, beta, gamma, wx, wy, wz, ox_dot, oy_dot, oz_dot= sym.symbols(r'o_x o_y o_z vx vy vz \alpha \beta \gamma w_x w_y w_z ox_{dot} oy_{dot} oz_{dot}')
tau_x, tau_y, tau_z, f_z = sym.symbols('tau_x, tau_y, tau_z, f_z')
m, Jx, Jy, Jz = sym.symbols('m J_x J_y J_z')
J_in1 = sym.diag(Jx, Jy, Jz)
tau_in1 = sym.Matrix([tau_x, tau_y, tau_z])
w_01in1 = sym.Matrix([wx, wy, wz])
# state: [o_x, o_x_dot, o_y, o_y_dot, o_z, o_z_dot, alpha, beta, gamma, wx, wy, wz]

# state: [o_x, vx, o_y, vy, o_z, vz, alpha, beta, gamma, wx, wy, wz]
# state_dot: [o_x_dot, vx_dot, o_y_dot, vy_dot, o_z_dot, vz_dot, alpha_dot, beta_dot, gamma_dot, wx_dot, wy_dot, wz_dot]

# inputs: [tau_x, tau_y, tau_z, f_z]
Rotation_matrix = Rotate(alpha, beta, gamma)
N = N_func(alpha, beta, gamma)
# second_derivatives = Rotation_matrix * sym.Matrix([0, 0, f_z])/m + sym.Matrix([0, 0, -g])
angle_derivatives = N * sym.Matrix([wx, wy, wz]) # [gamma_dot, beta_dot, alpha_dot]
w_derivatives = J_in1.inv() * (tau_in1 - w_01in1.cross(J_in1 * w_01in1))
f_in1 = Rotation_matrix.T * sym.Matrix([0, 0, -m * g]) + sym.Matrix([0, 0, f_z])
f_sym = sym.zeros(12,1)
# # f_sym[0] = ox_dot
# f_sym[1] = second_derivatives[0]
# # f_sym[2] = oy_dot
# f_sym[3] = second_derivatives[1]
# # f_sym[4] = oz_dot
# f_sym[5] = second_derivatives[2]
f_sym[6] = angle_derivatives[2]
f_sym[7] = angle_derivatives[1]
f_sym[8] = angle_derivatives[0]
f_sym[9] = w_derivatives[0]
f_sym[10] = w_derivatives[1]
f_sym[11] = w_derivatives[2]

# NEW CHANGES

v_01in1 = sym.Matrix([vx, vy, vz])
o_dot = Rotation_matrix @ v_01in1 # o_dot matrix in terms of vx,vy,vz
v_dot = (1 / m) * (f_in1 - w_01in1.cross(m * v_01in1))
f_sym[0] = o_dot[0]
f_sym[1] = v_dot[0]
f_sym[2] = o_dot[1]
f_sym[3] = v_dot[1]
f_sym[4] = o_dot[2]
f_sym[5] = v_dot[2]

f_sym

Matrix([
[           vx*cos(\alpha)*cos(\beta) + vy*(-sin(\alpha)*cos(\gamma) + sin(\beta)*sin(\gamma)*cos(\alpha)) + vz*(sin(\alpha)*sin(\gamma) + sin(\beta)*cos(\alpha)*cos(\gamma))],
[                                                                                                                                  (m*vy*w_z - m*vz*w_y + 9.81*m*sin(\beta))/m],
[            vx*sin(\alpha)*cos(\beta) + vy*(sin(\alpha)*sin(\beta)*sin(\gamma) + cos(\alpha)*cos(\gamma)) + vz*(sin(\alpha)*sin(\beta)*cos(\gamma) - sin(\gamma)*cos(\alpha))],
[                                                                                                                     (-m*vx*w_z + m*vz*w_x - 9.81*m*sin(\gamma)*cos(\beta))/m],
[                                                                                                       -vx*sin(\beta) + vy*sin(\gamma)*cos(\beta) + vz*cos(\beta)*cos(\gamma)],
[                                                                                                         

In [215]:
s = [o_x, vx, o_y, vy, o_z, vz, alpha, beta, gamma, wx, wy, wz]
i = [tau_x, tau_y, tau_z, f_z]
p = [m, Jx, Jy, Jz]
s_with_des = [o_x, o_y, o_z]

In [216]:
f = sym.lambdify(s + i + p, f_sym)
f

<function _lambdifygenerated(o_x, vx, o_y, vy, o_z, vz, Dummy_439, Dummy_438, _Dummy_437, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z, m, J_x, J_y, J_z)>

In [217]:
# Mass
m = 0.046

# Principle moments of inertia
J_x = 2.11e-05
J_y = 2.13e-05
J_z = 3.59e-05

# Acceleration of gravity
l_pen = 320e-3 # m

p_eq = [m, J_x, J_y, J_z]

In [218]:
s_eq = [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
i_eq = [0., 0., 0., g*(m)]

In [219]:
print(f(*s_eq, *i_eq, *p_eq))

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [220]:
A_sym = f_sym.jacobian(s)
B_sym = f_sym.jacobian(i)

A_num = sym.lambdify(s + i + p, A_sym)
B_num = sym.lambdify(s + i + p, B_sym)

A = A_num(*s_eq, *i_eq, *p_eq)
B = B_num(*s_eq, *i_eq, *p_eq)

In [221]:
# state: [o_x, o_x_dot, o_y, o_y_dot, o_z, o_z_dot, alpha, beta, gamma, wx, wy, wz, r, s, rdot, sdot]
############# JANK SHIT    ##############################
A_mod = np.vstack((np.hstack((A, np.zeros((12, 4)))), np.zeros((4,16))))
A_mod[12,14] = 1.0
A_mod[13,15] = 1.0
A_mod[14,12] = g/l_pen
A_mod[14, 7] = -g
A_mod[15,13] = g/l_pen
A_mod[15, 8] = g

B_mod = np.vstack((B, np.zeros((4,4))))

In [222]:

A_str = np.array2string(A_mod,
                        formatter={'float_kind': lambda x: f'{x:5.2f}'},
                        prefix='    ',
                        max_line_width=np.inf)

print(f'A = {A_str}')

A = [[ 0.00  1.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00 -0.00  0.00  9.81  0.00  0.00 -0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00 -0.00  0.00  0.00  0.00  0.00  0.00  0.00 -9.81  0.00  0.00 -0.00  0.00  0.00  0.00  0.00]
     [ 0.00 -0.00  0.00  0.00  0.00  1.00  0.00 -0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00 -0.00  0.00  0.00  0.00  0.00  0.00 -0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 -0.00 -0.00  1.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  

In [223]:
B_str = np.array2string(B_mod,
                        formatter={'float_kind': lambda x: f'{x:10.2f}'},
                        prefix='    ',
                        max_line_width=np.inf)
B
print(f'B = {B_str}')

B = [[      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00      21.74]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [  47393.36       0.00       0.00       0.00]
     [      0.00   46948.36       0.00       0.00]
     [      0.00       0.00   27855.15       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]]


In [259]:
Q = np.diag([
    100., # o_x
    1., # o_x_dot
    100., # o_y
    1., # o_y_dot
    1., # o_z
    1., # o_z_dot
    1., # alpha
    10., # beta
    10., # gamma
    100., # wx
    100., # wy
    1., # wz
    1., # r
    1., # s
    1., # rdot
    1.  # sdot
])

R = np.diag([
    8 * 1E7, #tau_x
    8 * 1E7,#tau_y
    2 * 1E7,
    2 * 100,
])

def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K


In [260]:
K = lqr(A_mod, B_mod, Q, R)

K

array([[-9.30968384e-15, -1.06647758e-14,  1.11803399e-03,
         1.47909239e-03, -6.97113333e-18, -8.91787679e-18,
         1.08292235e-17,  3.61743123e-14,  1.96994748e-02,
         1.44267732e-03,  5.61332134e-16,  7.66812039e-18,
        -3.05101511e-13,  9.13828116e-02, -5.51369706e-14,
         1.65049663e-02],
       [-1.11803399e-03, -1.47928683e-03, -1.85866453e-16,
         1.79512207e-16, -1.50533975e-17, -1.95607866e-17,
        -2.06554010e-16,  1.97231181e-02,  3.50883686e-14,
         5.56061410e-16,  1.44575407e-03, -1.23686423e-17,
        -9.14645832e-02,  1.15769861e-13, -1.65197347e-02,
         2.08969726e-14],
       [-5.44061589e-17, -1.04308110e-16, -3.24984446e-16,
        -2.16405515e-16, -7.85939486e-17,  4.77468664e-17,
         2.23606798e-04, -1.62971073e-15,  1.97305223e-16,
         1.80275588e-17, -2.93539922e-17,  2.57011611e-04,
         2.87543875e-15, -1.82610752e-15,  5.19997126e-16,
        -3.29225286e-16],
       [-6.23337188e-14, -7.04284134e

In [266]:
r_pos, rdot_pos, s_pos, sdot_pos = sym.symbols('r_pos, rdot_pos, s_pos, sdot_pos')
real_s = [o_x, vx, o_y, vy, o_z, vz, alpha, beta, gamma, wx, wy, wz, r_pos, s_pos, rdot_pos, sdot_pos]

In [267]:
def export_controller(K, s, i, s_with_des, i_eq,
                      decimals=8,
                      suffix='',
                      line_ending=''):
    """
    K is a gain matrix, of size m x n
    s is a list of states as symbolic variables, of length n
    i is a list of inputs as symbolic variables, of length m
    s_with_des is a list of states that have desired values, as
        symbolic variables - if there are no such states, then
        this should be an empty list []
    i_eq is a list of equilibrium values of inputs, of length m
    decimals is the number of decimals to include when printing
        each value
    suffix is the character (if any) to print after each number,
        for example 'f' to indicate a "float" when exporting to C
    line_ending is the character (if any) to print after each
        line, for example ';' when exporting to C
    """
    
    s_name = [scur.name for scur in s]
    s_name[6] = 'psi'
    s_name[7] = 'theta'
    s_name[8] = 'phi'
    i_name = [icur.name for icur in i]
    for row in range(len(i_name)):
        input_string = ''
        for col in range(len(s_name)):
            k = K[row, col]
            if not np.isclose(k, 0.):
                if (k < 0) and input_string:
                    input_string += ' +'
                if s[col] in s_with_des:
                    n = f'({s_name[col]} - {s_name[col]}_des)'
                else:
                    n = s_name[col]
                input_string += f' {-k:.{decimals}f}{suffix} * {n}'
        if not np.isclose(i_eq[row], 0.):
            if (i_eq[row] > 0) and input_string:
                input_string += ' +'
            input_string += f' {i_eq[row]:.{decimals}f}{suffix}'
        print(f'{i_name[row]} ={input_string}{line_ending}')


# # For python simulation
# export_controller(
#     K,               # the gain matrix
#     s,               # list of states as symbolic variables
#     i,               # list of inputs as symbolic variables
#     s_with_des,      # list of states that have desired values as symbolic variables
#     i_eq,            # list of equilibrium values of inputs
# )

# For C
export_controller(
    K,               # the gain matrix
    real_s,               #s list of states as symbolic variables
    i,               # list of inputs as symbolic variables
    s_with_des,      # list of states that have desired values as symbolic variables
    i_eq,            # list of equilibrium values of inputs
    suffix='f',      # character to print after each number (indicates a "float")
    line_ending=';'  # character to print after each line
)


tau_x = -0.00111803f * (o_y - o_y_des) -0.00147909f * vy -0.01969947f * phi -0.00144268f * w_x -0.09138281f * s_pos -0.01650497f * sdot_pos;
tau_y = 0.00111803f * (o_x - o_x_des) + 0.00147929f * vx -0.01972312f * theta -0.00144575f * w_y + 0.09146458f * r_pos + 0.01651973f * rdot_pos;
tau_z = -0.00022361f * psi -0.00025701f * w_z;
f_z = -0.07071068f * (o_z - o_z_des) -0.10726315f * vz + 0.45126000f;


In [268]:
l = 0.033
k_F = 2.78e-06
k_M = 7.50e-09

Defines the matrix $P$ that maps motor power commands ($m_1$, $m_2$, $m_3$, $m_4$) to inputs ($\tau_x$, $\tau_y$, $\tau_z$, $f_z$).

In [229]:
P = np.array([[ -l * k_F, -l * k_F,  l * k_F,  l * k_F  ],
              [ -l * k_F, l * k_F,   l * k_F,  -l * k_F ],
              [ -k_M,     k_M,       -k_M,     k_M      ],
              [ k_F,      k_F,       k_F,      k_F      ]])

Computes the matrix $P^{-1}$ that maps inputs to motor power commands.

In [230]:
Pinv = linalg.inv(P)

Print code that implements the method of power distribution in python (for simulation).

In [231]:
def export_power_distribution(Pinv,
                              limiter='self.limitUint16',
                              decimals=1,
                              suffix='',
                              line_ending=''):
    """
    Pinv is a 4 x 4 matrix that maps inputs (tau_x, tau_y, tau_z, f_z)
        to motor power commands (m_1, m_2, m_3, m_4)
    limiter is the name of the function to apply that ensures each
        motor power command is valid (i.e., an integer within bounds),
        for example "limitUint16" when exporting to C
    decimals is the number of decimals to include when printing
        each value
    suffix is the character (if any) to print after each number,
        for example 'f' to indicate a "float" when exporting to C
    line_ending is the character (if any) to print after each
        line, for example ';' when exporting to C
    """
    
    i_name = ['tau_x', 'tau_y', 'tau_z', 'f_z']
    m_name = ['m_1', 'm_2', 'm_3', 'm_4']
    for row in range(len(m_name)):
        input_string = ''
        for col in range(len(i_name)):
            k = Pinv[row, col]
            if not np.isclose(k, 0.):
                if (k > 0) and input_string:
                    input_string += ' +'
                n = i_name[col]
                input_string += f' {k:.{decimals}f}{suffix} * {n}'
        print(f'{m_name[row]} = {limiter}({input_string} ){line_ending}')

export_power_distribution(Pinv)

m_1 = self.limitUint16( -2725092.7 * tau_x -2725092.7 * tau_y -33333333.3 * tau_z + 89928.1 * f_z )
m_2 = self.limitUint16( -2725092.7 * tau_x + 2725092.7 * tau_y + 33333333.3 * tau_z + 89928.1 * f_z )
m_3 = self.limitUint16( 2725092.7 * tau_x + 2725092.7 * tau_y -33333333.3 * tau_z + 89928.1 * f_z )
m_4 = self.limitUint16( 2725092.7 * tau_x -2725092.7 * tau_y + 33333333.3 * tau_z + 89928.1 * f_z )


Print code that implements the method of power distribution in C (for hardware).

In [232]:
export_power_distribution(
    Pinv,
    limiter='limitUint16',
    suffix='f',
    line_ending=';',
)

m_1 = limitUint16( -2725092.7f * tau_x -2725092.7f * tau_y -33333333.3f * tau_z + 89928.1f * f_z );
m_2 = limitUint16( -2725092.7f * tau_x + 2725092.7f * tau_y + 33333333.3f * tau_z + 89928.1f * f_z );
m_3 = limitUint16( 2725092.7f * tau_x + 2725092.7f * tau_y -33333333.3f * tau_z + 89928.1f * f_z );
m_4 = limitUint16( 2725092.7f * tau_x -2725092.7f * tau_y + 33333333.3f * tau_z + 89928.1f * f_z );
